In [35]:
from cobra import Reaction, Metabolite
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

The metabolism of V. natriegens ATCC 14048 was investigated by Long et al. by carbon-13 flux analysis in order to determine intracellular metabolic fluxes of this organism (Long et al. 2017). These analyses were performed in a minimal media (M9) with glucose as an carbon source. For this reason, the refseq of V. natriegens ATCC 14048 from NCBI is chosen for the construction of the GSM in order to have as much simularity as possible to the isolate used. (TAKEN FROM GROUP7-2020)
We now switch to LB media to compare results. 

In [36]:
modelv=read_sbml_model('Models/Vibrio-natriegens-ATCC_14048-LB.xml')

In [37]:
print("Number of reactions:", len(modelv.reactions))
print("Number of metabolites:", len(modelv.metabolites))
print("Number of genes included in the model:", len(modelv.genes))
# Percentage of coding genes covered (from: https://www.ncbi.nlm.nih.gov/nuccore/520916752?report=genbank)
print("% of genes in model compared to the number of coding genes in Vibrio natrigens(4513 genes):",((len(modelv.genes)*100)/4513))

Number of reactions: 2720
Number of metabolites: 1730
Number of genes included in the model: 1480
% of genes in model compared to the number of coding genes in Vibrio natrigens(4513 genes): 32.7941502326612


In [69]:
print("Maximal growth rate of Vibrio natriegens in LB medium", modelv.optimize().objective_value)

Maximal growth rate of Vibrio natriegens in LB medium 2.8132423768018597


First, one has to look for the names corresponding to the metabolites belonging to the violacein pathway.

In [39]:
modelv.reactions.TRPS2 # Reaction before L-trp to IPA Imine!

Reaction identifier,TRPS2
Name,Tryptophan synthase (indole)
Memory address,0x01fc44814eb8
Stoichiometry,indole_c + ser__L_c --> h2o_c + trp__L_c Indole + L-Serine --> H2O H2O + L-Tryptophan
GPR,WP_217999032_1 or ( WP_014232411_1 and WP_020335573_1 )
Lower bound,0.0
Upper bound,1000.0


After finding the starting point of the violacein pathway, one has to make sure that the corresponding reactions to level Chlorines, hemo groups and pyruvate exist in this model. It is ensured that the model is properly balanced, and the species would not die. The side reactions can be found in *Side_reactions.ipynb*.

Once everything is properly balanced, we start adding violacein production pathway.

We define all metabolites used that are not included in *Vibrio Natrigens* model.

In [40]:
# --- For violacein production ---
ipa_c=Metabolite(id='ipa_c', formula='C11H10N2O2', name='2-Imino-3-(indol-3-yl)propanoate', compartment='c')
ipa_dimer_c=Metabolite(id='ipa_dimer_c', compartment='c')
ptdv_c=Metabolite(id='ptdv_c', formula='C21H15N3O2', name='Protodeoxyviolaceinate', compartment='c')
ptv_c=Metabolite(id='ptv_c', formula='C21H15N3O3', name='Protoviolaceinate', compartment='c')
va_c=Metabolite(id='va_c', formula='C21H15N3O4', name='Violaceinate', compartment='c')
v_c=Metabolite(id='v_c', formula='C20H13N3O3', name='Violacein', compartment='c')
# --- Side reactions ---
cp_c=Metabolite(id='cp_c', formula='C22H15N3O4', name='Chromopyrrolate', compartment='c')
dva_c=Metabolite(id='dva_c', formula='C21H15N3O3', name='Deoxyviolaceinate', compartment='c')
dv_c=Metabolite(id='dv_c', formula='C20H13N3O2', name='Deoxyviolacein', compartment='c')
pdv_c=Metabolite(id='pdv_c',formula='C20H13N3O', name='Prodeoxyviolacein', compartment='c')
pv_c=Metabolite(id='pv_c',formula='C20H13N3O2', name='Proviolacein', compartment='c')

We build up every reaction from tryptophan to violacein.

In [41]:
Reaction_VioA_FAD=Reaction('IPA_imine_VioA_FAD') # L-tryptophan + FAD <=> IPA imine + FADH2

Reaction_VioA_FAD.add_metabolites({modelv.metabolites.trp__L_c: -1, # L-tryptophan
                                 modelv.metabolites.fad_c: -1, # FAD
                                 ipa_c: 1, # IPA imine
                                 modelv.metabolites.fadh2_c: 1 # FADH2
                                })

In [42]:
Reaction_VioA_O2=Reaction('IPA_imine_VioA_O2') # L-Tryptophan + Oxygen <=> IPA imine + Hydrogen peroxide

Reaction_VioA_O2.add_metabolites({modelv.metabolites.trp__L_c: -1,   # L-tryptophan
                                 modelv.metabolites.o2_c: -1,            # Oxygen
                                 ipa_c: 1,                               # IPA imine
                                 modelv.metabolites.h2o2_c: 1            # Hydrogen peroxide
                                })

In [43]:
Reaction_VioB=Reaction('IPA_imine_dimer_VioB') #  # 2 IPA imine + Fe2+ => IPA imine dimer + Fe3+ 

Reaction_VioB.add_metabolites({ipa_c: -2,               # IPA imine
                                 modelv.metabolites.fe2_c: -1,    # Fe2+
                                 ipa_dimer_c: 1,                  # IPA imine dimer
                                 modelv.metabolites.fe3_c: 1      # Fe3+ 
                                })

In [44]:
Reaction_Chromopyrrolate=Reaction('Chromopyrrolate') # IPA imine dimer <=> Chromopyrollic acid + NH3 

Reaction_Chromopyrrolate.add_metabolites({ipa_dimer_c: -1,         # IPA imine
                                 cp_c: 1,                   # Chromopyrollic acid
                                 modelv.metabolites.nh3_c: 1       # NH3 
                                })

In [45]:
Reaction_VioE=Reaction('Protodeoxyviolaceinate_VioE') # IPA imine dimer <=> Protodeoxyviolaceinic acid + CO2 + NH3

Reaction_VioE.add_metabolites({ipa_dimer_c: -1,      # IPA imine dimer
                                 modelv.metabolites.co2_c: 1,          # CO2
                                 ptdv_c: 1,                            # Protodeoxyviolaceinic acid
                                 modelv.metabolites.nh3_c: 1           # NH3 
                                })

In [46]:
Reaction_Prodeoxyviolacein=Reaction('Prodeoxyviolacein') # Protodeoxyviolaceinic acid + O2 <=> Prodeoxyviolacein + CO2 +H2O

Reaction_Prodeoxyviolacein.add_metabolites({ptdv_c: -1,                # Protodeoxyviolaceinic acid
                                 modelv.metabolites.o2_c: -1,          # Oxygen         
                                 pdv_c: 1,                             # Prodeoxyviolacein
                                 modelv.metabolites.co2_c: 1,          # CO2
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [47]:
Reaction_Deoxyviolaceinate_VioC=Reaction('Deoxyviolaceinate_VioC') # Protodeoxyviolaceinic acid + Oxygen + NADPH + H+ <=> Deoxyviolaceinic acid + NADP+ + H2O

Reaction_Deoxyviolaceinate_VioC.add_metabolites({ptdv_c: -1,           # Protodeoxyviolaceinic acid
                                 modelv.metabolites.nadph_c: -1,       # NADPH
                                 modelv.metabolites.o2_c: -1,          # Oxygen
                                 modelv.metabolites.h_c: -1,           # H+           
                                 dva_c: 1,                             # Deoxyviolaceinic acid
                                 modelv.metabolites.nadp_c: 1,         # NADP+
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [48]:
Reaction_Deoxyviolacein=Reaction('Deoxyviolacein') # Deoxyviolaceinic acid + Oxygen <=> Deoxyviolacein + CO2 + H2O

Reaction_Deoxyviolacein.add_metabolites({dva_c: -1,                    # Deoxyviolaceinic acid
                                 modelv.metabolites.o2_c: -1,          # Oxygen         
                                 dv_c: 1,                              # Deoxyviolacein
                                 modelv.metabolites.co2_c: 1,          # CO2
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [49]:
Reaction_VioD=Reaction('Protoviolaceinate_VioD') # Protodeoxyviolaceinic acid + NADPH + Oxygen + H+ <=> Protoviolaceinic acid + NADP+ + H2O

Reaction_VioD.add_metabolites({ptdv_c: -1,                             # Protodeoxyviolaceinic acid
                                 modelv.metabolites.nadph_c: -1,       # NADPH
                                 modelv.metabolites.o2_c: -1,          # Oxygen
                                 modelv.metabolites.h_c: -1,           # H+           
                                 ptv_c: 1,                             # Protoviolaceinic acid
                                 modelv.metabolites.nadp_c: 1,         # NADP+
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [50]:
Reaction_Proviolacein=Reaction('Proviolacein') # Protoviolaceinic acid + O2 = Proviolacein +CO2 + H2O

Reaction_Proviolacein.add_metabolites({ptv_c: -1,                    # Protoviolaceinic acid
                                 modelv.metabolites.o2_c: -1,          # Oxygen         
                                 pv_c: 1,                              # Proviolacein
                                 modelv.metabolites.co2_c: 1,          # CO2
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [51]:
Reaction_Violaceinate_VioC=Reaction('Violaceinate') # Protoviolaceinic acid + Oxygen + NADPH + H+ <=> Violaceinic acid + NADP+ + H2O

Reaction_Violaceinate_VioC.add_metabolites({ptv_c: -1,                 # Protoviolaceinic acid
                                 modelv.metabolites.nadph_c: -1,       # NADPH
                                 modelv.metabolites.o2_c: -1,          # Oxygen
                                 modelv.metabolites.h_c: -1,           # H+           
                                 va_c: 1,                              # Violaceinic acid
                                 modelv.metabolites.nadp_c: 1,         # NADP+
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [52]:
Reaction_VioD=Reaction('Protoviolaceinate_VioD') # Protodeoxyviolaceinic acid + NADPH + Oxygen + H+ <=> Protoviolaceinic acid + NADP+ + H2O

Reaction_VioD.add_metabolites({ptdv_c: -1,                             # Protodeoxyviolaceinic acid
                                 modelv.metabolites.nadph_c: -1,       # NADPH
                                 modelv.metabolites.o2_c: -1,          # Oxygen
                                 modelv.metabolites.h_c: -1,           # H+           
                                 ptv_c: 1,                             # Protoviolaceinic acid
                                 modelv.metabolites.nadp_c: 1,         # NADP+
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [53]:
Reaction_Proviolacein=Reaction('Proviolacein') # Protoviolaceinic acid + O2 = Proviolacein +CO2 + H2O

Reaction_Proviolacein.add_metabolites({ptv_c: -1,                    # Protoviolaceinic acid
                                 modelv.metabolites.o2_c: -1,          # Oxygen         
                                 pv_c: 1,                              # Proviolacein
                                 modelv.metabolites.co2_c: 1,          # CO2
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [54]:
Reaction_Violaceinate_VioC=Reaction('Violaceinate') # Protoviolaceinic acid + Oxygen + NADPH + H+ <=> Violaceinic acid + NADP+ + H2O

Reaction_Violaceinate_VioC.add_metabolites({ptv_c: -1,                 # Protoviolaceinic acid
                                 modelv.metabolites.nadph_c: -1,       # NADPH
                                 modelv.metabolites.o2_c: -1,          # Oxygen
                                 modelv.metabolites.h_c: -1,           # H+           
                                 va_c: 1,                              # Violaceinic acid
                                 modelv.metabolites.nadp_c: 1,         # NADP+
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

In [55]:
Reaction_Violacein=Reaction('Violacein') # Violaceinic acid + Oxygen <=> Violacein + CO2 + H2O

Reaction_Violacein.add_metabolites({va_c: -1,                          # Violaceinic acid
                                 modelv.metabolites.o2_c: -1,          # Oxygen         
                                 v_c: 1,                               # Violacein
                                 modelv.metabolites.co2_c: 1,          # CO2
                                 modelv.metabolites.h2o_c: 1           # H2O
                                })

All reactions must be added to the model.

In [56]:
# modelv.add_reactions([Reaction_VioA_O2]) or modelv.add_reactions([Reaction_VioA_FAD]) WE SHOULD DECIDE WHICH TO USE!!!!!
modelv.add_reactions([Reaction_VioA_O2])
modelv.add_reactions([Reaction_VioB])
modelv.add_reactions([Reaction_Chromopyrrolate])
modelv.add_reactions([Reaction_VioE])
modelv.add_reactions([Reaction_Prodeoxyviolacein])
modelv.add_reactions([Reaction_Deoxyviolaceinate_VioC])
modelv.add_reactions([Reaction_Deoxyviolacein])
modelv.add_reactions([Reaction_VioD])
modelv.add_reactions([Reaction_Proviolacein])
modelv.add_reactions([Reaction_Violaceinate_VioC])
modelv.add_reactions([Reaction_Violacein])

The new model including violacein pathway is stored in Models folder.

In [57]:
# Idented to avoid writing over the file every time. At the end of the project we should unident it :)

# write_sbml_model(modelv,"Models/model_violacein_o2.xml")

The following step one must do is changing the objective of the model.

Making sure boundaries are set correctly:

In [58]:
modelv.reactions.Violacein.bounds

(0.0, 1000.0)

In [59]:
modelv.reactions.Deoxyviolacein.bounds

(0.0, 1000.0)

In [60]:
modelv.reactions.Prodeoxyviolacein.bounds

(0.0, 1000.0)

In [61]:
modelv.reactions.Proviolacein.bounds

(0.0, 1000.0)

What metabolites are sinks or demands?

In [62]:
modelv.sinks

[<Reaction sink_2ohph_c at 0x1fc44982dd8>,
 <Reaction sink_4crsol_c at 0x1fc44982518>,
 <Reaction sink_4hba_c at 0x1fc44982b38>,
 <Reaction sink_aacald_c at 0x1fc44990780>,
 <Reaction sink_amob_c at 0x1fc44990080>,
 <Reaction sink_hemeO_c at 0x1fc44990668>,
 <Reaction sink_mobd_c at 0x1fc44990898>,
 <Reaction sink_sheme_c at 0x1fc449904e0>]

In [63]:
modelv.demands

[]

Setting the type of boundaries to "demand" for Violacein, Deoxyviolacein, Prodeoxyviolacein and Proviolacein:

In [64]:
modelv.add_boundary(modelv.metabolites.get_by_id("v_c"), type="demand")

Reaction identifier,DM_v_c
Name,Violacein demand
Memory address,0x01fc44990400
Stoichiometry,v_c --> Violacein -->
GPR,
Lower bound,0
Upper bound,1000.0


In [65]:
modelv.add_boundary(modelv.metabolites.get_by_id("pdv_c"), type="demand")

Reaction identifier,DM_pdv_c
Name,Prodeoxyviolacein demand
Memory address,0x01fc44a4c198
Stoichiometry,pdv_c --> Prodeoxyviolacein -->
GPR,
Lower bound,0
Upper bound,1000.0


In [66]:
modelv.add_boundary(modelv.metabolites.get_by_id("dv_c"), type="demand")

Reaction identifier,DM_dv_c
Name,Deoxyviolacein demand
Memory address,0x01fc44a4c4a8
Stoichiometry,dv_c --> Deoxyviolacein -->
GPR,
Lower bound,0
Upper bound,1000.0


In [67]:
modelv.add_boundary(modelv.metabolites.get_by_id("pv_c"), type="demand")

Reaction identifier,DM_pv_c
Name,Proviolacein demand
Memory address,0x01fc44a3db70
Stoichiometry,pv_c --> Proviolacein -->
GPR,
Lower bound,0
Upper bound,1000.0


In [68]:
with modelv:
    modelv.objective = modelv.reactions.Violacein
    print(modelv.optimize)
    violacein_production = modelv.optimize().objective_value
    
    print('Max. violacein production [mmol gDW^-1 h^-1]:',violacein_production)

<bound method Model.optimize of <Model Vibrio_natriegens_ATCC_14048_LB at 0x1fc427fc748>>
Max. violacein production [mmol gDW^-1 h^-1]: 2.5


The maximum production rate of violacein in *V. natriegens* with the violacein pathway from *V. campbellii* is 1.629 mmol gDW-1 h-1